In [7]:
from tfg.alphaZero import create_alphazero, AlphaZero
from tfg.alphaZeroCallbacks import Checkpoint
from tfg.util import enable_gpu, play
from tfg.strategies import Minimax, MonteCarloTree
from game.tictactoe import TicTacToe

In [2]:
enable_gpu()

game = TicTacToe()

In [5]:
callback = Checkpoint('exp_chk', 'models/checkpoints', delay=4)

alphazero = create_alphazero(game, max_workers=8, self_play_times=64,
                             max_games_counter=2048, buffer_size=1024,
                             callbacks=[callback], mcts_iter=100)

Epoch 1/10
4/4 - 2s - loss: 1.5525 - value_head_loss: 0.5437 - policy_head_loss: 2.4106
Epoch 2/10
4/4 - 0s - loss: 1.3929 - value_head_loss: 0.4252 - policy_head_loss: 2.1584
Epoch 3/10
4/4 - 0s - loss: 1.3886 - value_head_loss: 0.4113 - policy_head_loss: 2.1158
Epoch 4/10
4/4 - 0s - loss: 1.3449 - value_head_loss: 0.3540 - policy_head_loss: 2.0492
Epoch 5/10
4/4 - 0s - loss: 1.3295 - value_head_loss: 0.3464 - policy_head_loss: 1.9999
Epoch 6/10
4/4 - 0s - loss: 1.2781 - value_head_loss: 0.3017 - policy_head_loss: 1.9240
Epoch 7/10
4/4 - 0s - loss: 1.2281 - value_head_loss: 0.2798 - policy_head_loss: 1.8343
Epoch 8/10
4/4 - 0s - loss: 1.1916 - value_head_loss: 0.2851 - policy_head_loss: 1.7490
Epoch 9/10
4/4 - 0s - loss: 1.1448 - value_head_loss: 0.2577 - policy_head_loss: 1.6789
Epoch 10/10
4/4 - 0s - loss: 1.1034 - value_head_loss: 0.2419 - policy_head_loss: 1.6098
Games played: 64
Epoch 1/10
4/4 - 0s - loss: 1.2599 - value_head_loss: 0.3167 - policy_head_loss: 1.8475
Epoch 2/10
4/4

Epoch 3/10
4/4 - 0s - loss: 0.6835 - value_head_loss: 0.2064 - policy_head_loss: 0.9623
Epoch 4/10
4/4 - 0s - loss: 0.6579 - value_head_loss: 0.1852 - policy_head_loss: 0.9318
Epoch 5/10
4/4 - 0s - loss: 0.6378 - value_head_loss: 0.1742 - policy_head_loss: 0.9022
Epoch 6/10
4/4 - 0s - loss: 0.6204 - value_head_loss: 0.1549 - policy_head_loss: 0.8872
Epoch 7/10
4/4 - 0s - loss: 0.6043 - value_head_loss: 0.1344 - policy_head_loss: 0.8767
Epoch 8/10
4/4 - 0s - loss: 0.5904 - value_head_loss: 0.1277 - policy_head_loss: 0.8576
Epoch 9/10
4/4 - 0s - loss: 0.5854 - value_head_loss: 0.1254 - policy_head_loss: 0.8528
Epoch 10/10
4/4 - 0s - loss: 0.5634 - value_head_loss: 0.1057 - policy_head_loss: 0.8318
Games played: 640
Epoch 1/10
4/4 - 0s - loss: 0.8493 - value_head_loss: 0.3330 - policy_head_loss: 1.1800
Epoch 2/10
4/4 - 0s - loss: 0.7221 - value_head_loss: 0.2354 - policy_head_loss: 1.0249
Epoch 3/10
4/4 - 0s - loss: 0.6910 - value_head_loss: 0.1998 - policy_head_loss: 0.9982
Epoch 4/10
4/

Epoch 5/10
4/4 - 0s - loss: 0.5740 - value_head_loss: 0.2205 - policy_head_loss: 0.8143
Epoch 6/10
4/4 - 0s - loss: 0.5626 - value_head_loss: 0.1957 - policy_head_loss: 0.8162
Epoch 7/10
4/4 - 0s - loss: 0.5486 - value_head_loss: 0.1828 - policy_head_loss: 0.8013
Epoch 8/10
4/4 - 0s - loss: 0.5351 - value_head_loss: 0.1718 - policy_head_loss: 0.7855
Epoch 9/10
4/4 - 0s - loss: 0.5213 - value_head_loss: 0.1472 - policy_head_loss: 0.7829
Epoch 10/10
4/4 - 0s - loss: 0.5222 - value_head_loss: 0.1512 - policy_head_loss: 0.7815
Games played: 1216
Epoch 1/10
4/4 - 0s - loss: 0.6785 - value_head_loss: 0.2718 - policy_head_loss: 0.9745
Epoch 2/10
4/4 - 0s - loss: 0.6097 - value_head_loss: 0.1960 - policy_head_loss: 0.9122
Epoch 3/10
4/4 - 0s - loss: 0.5741 - value_head_loss: 0.1757 - policy_head_loss: 0.8588
Epoch 4/10
4/4 - 0s - loss: 0.5445 - value_head_loss: 0.1588 - policy_head_loss: 0.8134
Epoch 5/10
4/4 - 0s - loss: 0.5331 - value_head_loss: 0.1467 - policy_head_loss: 0.8003
Epoch 6/10
4

Epoch 7/10
4/4 - 0s - loss: 0.4439 - value_head_loss: 0.1077 - policy_head_loss: 0.6755
Epoch 8/10
4/4 - 0s - loss: 0.4457 - value_head_loss: 0.1083 - policy_head_loss: 0.6802
Epoch 9/10
4/4 - 0s - loss: 0.4334 - value_head_loss: 0.0985 - policy_head_loss: 0.6674
Epoch 10/10
4/4 - 0s - loss: 0.4271 - value_head_loss: 0.0929 - policy_head_loss: 0.6625
Games played: 1792
Epoch 1/10
4/4 - 0s - loss: 0.5613 - value_head_loss: 0.2223 - policy_head_loss: 0.8036
Epoch 2/10
4/4 - 0s - loss: 0.4822 - value_head_loss: 0.1486 - policy_head_loss: 0.7202
Epoch 3/10
4/4 - 0s - loss: 0.4570 - value_head_loss: 0.1246 - policy_head_loss: 0.6930
Epoch 4/10
4/4 - 0s - loss: 0.4457 - value_head_loss: 0.1172 - policy_head_loss: 0.6766
Epoch 5/10
4/4 - 0s - loss: 0.4342 - value_head_loss: 0.1069 - policy_head_loss: 0.6632
Epoch 6/10
4/4 - 0s - loss: 0.4215 - value_head_loss: 0.0962 - policy_head_loss: 0.6485
Epoch 7/10
4/4 - 0s - loss: 0.4156 - value_head_loss: 0.0898 - policy_head_loss: 0.6437
Epoch 8/10
4

In [5]:
alphazero = AlphaZero(game, buffer_size=1024, mcts_iter=100)

In [6]:
alphazero.load('models/experiment_tictactoe.h5')

In [8]:
mcts_draws = list()
az_draws = list()
minimax = Minimax(game)
iters = (2, 10, 50, 100, 200, 400)

for m in iters:
    mcts = MonteCarloTree(game, max_iter=m, reset_tree=False)
    _, draws, _ = play(game, minimax, mcts, games=100, max_workers=10)
    mcts_draws.append(draws)
    
    alphazero.set_max_iter(m)
    _, draws, _ = play(game, minimax, alphazero, games=100)
    az_draws.append(draws)

mcts_draws, az_draws

([0, 9, 35, 53, 60, 80], [79, 90, 86, 87, 85, 92])

In [25]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x=iters, y=mcts_draws, mode='lines+markers', name='MCTS draws'))
fig.add_trace(go.Scatter(x=iters, y=az_draws, mode='lines+markers', name='AZ draws'))
fig.update_xaxes(title_text='Iterations')
fig.update_yaxes(title_text='Draws')
fig.show()

In [35]:
from tfg.strategies import HumanStrategy


alphazero.set_max_iter(1000)

play(game, HumanStrategy(game), alphazero, render=True, print_results=True)

 | | 
-+-+-
 | | 
-+-+-
 | | 


[PLAYER] choose action: 0, 1, 2, 3, 4, 5, 6, 7, 8
> 4
 | | 
-+-+-
 |X| 
-+-+-
 | | 

 |O| 
-+-+-
 |X| 
-+-+-
 | | 


[PLAYER] choose action: 0, 2, 3, 5, 6, 7, 8
> 0
X|O| 
-+-+-
 |X| 
-+-+-
 | | 

X|O| 
-+-+-
O|X| 
-+-+-
 | | 


[PLAYER] choose action: 2, 5, 6, 7, 8
> 8
X|O| 
-+-+-
O|X| 
-+-+-
 | |X

PLAYER 1 WON


(1, 0, 0)

In [36]:
model = alphazero.neural_network.model

In [46]:
import numpy as np
board = np.array([[
    [[0, 0, 0],
     [0, 1, 0],
     [0, 0, 0]],
    [[0, 0, 0],
     [0, 0, 0],
     [0, 0, 0]],
    [[1] * 3] * 3
]])
v, p = model(board)
v = v.numpy()[0, 0]
p = p.numpy()[0]
v, p, np.argmax(p)

(0.05857558,
 array([8.41069698e-01, 9.47743138e-06, 1.65077136e-03, 4.35717553e-02,
        1.02105215e-01, 1.15720062e-02, 3.81052564e-06, 6.27960119e-08,
        1.71607226e-05], dtype=float32),
 0)